In [1]:
import numpy as np
import pandas as pd
import nltk
import csv

In [87]:
import numpy as np
import pandas as pd
import nltk
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import requests
import json
import re

In [2]:
df = pd.read_csv('lipstick.csv')

In [3]:
df[50:58]

,Unnamed: 0,product_name,brand_name,price,love_count,color,rgb,link
50,50,Matte Revolution Lipstick,CHARLOTTE TILBURY,34.0,55206,Glastonberry muted purple berry,"(145, 76, 121)",https://www.sephora.com/product/matte-revoluti...
51,51,Matte Revolution Lipstick,CHARLOTTE TILBURY,34.0,55206,Love Liberty berry pink,"(163, 33, 97)",https://www.sephora.com/product/matte-revoluti...
52,52,Gloss Bomb Universal Lip Luminizer,FENTY BEAUTY BY RIHANNA,18.0,256222,Diamond Milk shimmering pearl,"(223, 204, 197)",https://www.sephora.com/product/gloss-bomb-uni...
53,53,Gloss Bomb Universal Lip Luminizer,FENTY BEAUTY BY RIHANNA,18.0,256222,Fenty Glow shimmering rose nude,"(181, 105, 91)",https://www.sephora.com/product/gloss-bomb-uni...
54,54,Velvet Matte Lipstick Pencil,NARS,27.0,309898,Get Off pink beige,"(243, 160, 146)",https://www.sephora.com/product/velvet-matte-l...
55,55,Velvet Matte Lipstick Pencil,NARS,27.0,309898,Intriguing rose pink,"(241, 139, 150)",https://www.sephora.com/product/velvet-matte-l...
56,56,Velvet Matte Lipstick Pencil,NARS,27.0,309898,Roman Holiday vibrant pink sheen,"(252, 159, 180)",https://www.sephora.com/product/velvet-matte-l...
57,57,Velvet Matte Lipstick Pencil,NARS,27.0,309898,Out of stock: Do Me Baby chestnut rose,"(217, 129, 151)",https://www.sephora.com/product/velvet-matte-l...


In [4]:
# df = df[ ~ df['product_name'].str.contains('Set') ]
invalid_info = df[df['price'].isna()]['price'].values
df = df[-df['price'].isin(invalid_info)]
invalid_info = df[df['love_count'].isna()]['love_count'].values
df = df[-df['love_count'].isin(invalid_info)]
df = df[ ~ df['product_name'].str.contains('Set') ]
df = df.reset_index(drop=True)

In [5]:
df.head()

,Unnamed: 0,product_name,brand_name,price,love_count,color,rgb,link
0,0,Lip Maestro,GIORGIO ARMANI BEAUTY,38.0,103038,Out of stock: 405,"(141, 6, 2)",https://www.sephora.com/product/lip-maestro-P3...
1,1,Lip Maestro,GIORGIO ARMANI BEAUTY,38.0,103038,507,"(144, 75, 94)",https://www.sephora.com/product/lip-maestro-P3...
2,2,Lip Maestro,GIORGIO ARMANI BEAUTY,38.0,103038,406,"(137, 38, 33)",https://www.sephora.com/product/lip-maestro-P3...
3,3,Lip Maestro,GIORGIO ARMANI BEAUTY,38.0,103038,508,"(245, 161, 151)",https://www.sephora.com/product/lip-maestro-P3...
4,4,Lip Maestro,GIORGIO ARMANI BEAUTY,38.0,103038,Out of stock: 400,"(163, 25, 23)",https://www.sephora.com/product/lip-maestro-P3...


In [6]:
df = df.sort_values(by='love_count', ascending=False)
df = df.reset_index(drop=True)

In [40]:
# df['slicing'] = [len(i) for i in df['brand_name']]

In [7]:
for i in range(len(df['color'])):
    if 'Out of stock: ' in str(df['color'].iloc[i]):
        df['color'].iloc[i] = df['color'].iloc[i][len('Out of stock: '): ]



C:\Users\Public\anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
df.head()

,Unnamed: 0,product_name,brand_name,price,love_count,color,rgb,link
0,107,Everlasting Liquid Lipstick,KAT VON D,20.0,1148262,Muñeca nude petal,"(228, 108, 110)",https://www.sephora.com/product/everlasting-lo...
1,125,Everlasting Liquid Lipstick,KAT VON D,20.0,1148262,Santa Sangre poison apple,"(208, 58, 59)",https://www.sephora.com/product/everlasting-lo...
2,115,Everlasting Liquid Lipstick,KAT VON D,20.0,1148262,Hawkwind nude sienna,"(157, 64, 74)",https://www.sephora.com/product/everlasting-lo...
3,116,Everlasting Liquid Lipstick,KAT VON D,20.0,1148262,Sanctuary nude coffee,"(142, 79, 87)",https://www.sephora.com/product/everlasting-lo...
4,117,Everlasting Liquid Lipstick,KAT VON D,20.0,1148262,Crucifix deep chocolate,"(114, 66, 64)",https://www.sephora.com/product/everlasting-lo...


In [86]:
def popular_lipstick(df):
    popular_lipstick = list()
    df_popular = df.groupby(['product_name','brand_name','love_count']).count().sort_values(by='love_count', ascending=False)
    df_popular.reset_index(inplace=True)
    for i in range(5):
        popular_lipstick.append((df_popular['product_name'].iloc[i], df_popular['brand_name'].iloc[i], df_popular['price'].iloc[i]))
    return popular_lipstick
popular_lipstick = popular_lipstick(df)
print( 'The top 5 loved lipsticks are:') 
popular_lipstick

The top 5 loved lipsticks are:


[('Everlasting Liquid Lipstick', 'KAT VON D', 32),
 ('Liquid Lipstick', 'ANASTASIA BEVERLY HILLS', 32),
 ('Cream Lip Stain Liquid Lipstick', 'SEPHORA COLLECTION', 12),
 ('Vice Lipstick', 'URBAN DECAY', 3),
 ('Full-On™ Plumping Lip Polish Gloss', 'BUXOM', 10)]

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [ ]:
!pip install selenium

# Function for info

In [ ]:
def get_lipstick_reviews(link):
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    import time
    lipstick_reviews = dict()
    options = webdriver.ChromeOptions()
    prefs = {
        'profile.default_content_setting_values' : {
            'images' : 2
        }
    }
    options.add_experimental_option('prefs',prefs)
    driver = webdriver.Chrome(options = options)
    driver.get(link)
    driver.find_element_by_class_name("css-1n39fc0 ").click()
    js="var q=document.documentElement.scrollTop=1000" 
    driver.execute_script(js)
    time.sleep(1)
    try:
        rating = float(driver.find_element_by_class_name("css-ffj77u ").text.split('/')[0].strip())
    except:
        rating = None
    try:
        reco_percent = int(driver.find_element_by_class_name("css-vrylf1 ").text.replace('%','').strip())
    except:
        reco_percent = None
    reviews = list()
    while True:
        try:
            driver.find_element_by_class_name("css-2wz498 ").click()
        except:
            break
    while True:
        try:
            if driver.find_element_by_xpath("//div[@data-comp = 'Review']"):
                for elem in driver.find_elements(By.XPATH, "//div[@data-comp = 'Review']"):
                    try:
                        user_info = elem.find_element_by_class_name("css-1pgaz2s ").text.strip()
                    except:
                        user_info = None
                    user_rating = int(elem.find_element_by_class_name("css-17ol29l ").get_attribute('style').split(':')[1].replace('%;','').strip()) / 20
                    try:
                        user_color = elem.find_element_by_class_name("css-7b8onn ").text.split(':')[1].strip()
                    except:
                        user_color = None
                    try:
                        elem.find_element_by_class_name("css-10o012f ").click()
                        user_review = elem.find_element_by_class_name("css-eq4i08 ").text.strip()
                    except:
                        try:
                            user_review = elem.find_element_by_class_name("css-eq4i08 ").text.strip()
                        except:
                            user_review = None
                    reviews.append({'user_info': user_info,
                                    'user_rating': user_rating,
                                    'user_color': user_color,
                                    'user_review': user_review})
                break
            else:
                break
        except:
            break
    driver.close()
    lipstick_reviews['rating'] = rating
    lipstick_reviews['reco_percent'] = reco_percent
    lipstick_reviews['reviews'] = reviews
    return lipstick_reviews

# Function for link

In [ ]:
def search_link(brand_name, product_name, color = ''):
    base_url = 'https://www.sephora.com/search?keyword='
    search_url = brand_name + ' ' + product_name + ' ' + color
    url = base_url + search_url
    return url

In [ ]:
# https://www.sephora.com/product/lip-maestro-P393411?skuId=1441591&keyword=giorio%20armani%20lip%20maestro%20401
product_name = 'Rouge Pur Couture The Slim Matte Lipstick'
brand_name = 'Yves Saint Laurent'
color = '1 Rouge Extravagant bright red'
link = search_link(brand_name, product_name, color)
link

In [ ]:
outcome = get_lipstick_reviews(link)

# Function for wordcloud

In [ ]:
def get_wordcloud(outcome):
    texts = []
    for i in outcome['reviews']:
        review = i['user_review']
        texts.append(review)
    # outcome['reviews']
    text_string = ''
    text_string_2 = ''
    from wordcloud import WordCloud, STOPWORDS
    import matplotlib.pyplot as plt
    %matplotlib inline
    #Remove unwanted words
    DELETE_WORDS = ['lipstick', 'lip', 'idea', 'product', 'the']
    def remove_words(text_string,DELETE_WORDS=DELETE_WORDS):
        for word in DELETE_WORDS:
            text_string = text_string.replace(word,' ')
        return text_string

    #Remove short words
    MIN_LENGTH = 0
    def remove_short_words(text_string,min_length = MIN_LENGTH):
        word_list = text_string.split()
        for word in word_list:
            if len(word) < min_length:
                text_string = text_string.replace(' '+word+' ',' ',1)
        return text_string



    for i in texts:
        string = remove_words(i)
        text_string = text_string + string

    from nltk.corpus import stopwords
    import nltk
    from nltk.stem.porter import PorterStemmer
    from nltk.tokenize import word_tokenize
    from nltk.tokenize import sent_tokenize
    words = word_tokenize(text_string)
    text_string = [word.lower() for word in words
                      if word not in stopwords.words() and word.isalpha()]
    text_string_ = ''
    for i in text_string:
        text_string_ = text_string_ + ' ' + i

#     p_stemmer = PorterStemmer()
#     # texts = inaugural.raw()
#     striptext = text_string_.replace('\n\n', ' ')
#     striptext = striptext.replace('\n', ' ')
#     sentences = sent_tokenize(striptext)
#     words = word_tokenize(striptext)
#     text_string_ = nltk.Text([p_stemmer.stem(i).lower() for i in words])

#     for i in text_string_:
#         if len(i) > 1:
#             if i[-1] == 'i':
#                 i = i[:-1] + 'y'
#         text_string_2 = text_string_2 +' '+ i
#     text_string_new = ''
#     for i in text_string_2:
#         text_string_new = text_string_new + ' ' + i
#     text_string_new
    from wordcloud import WordCloud, STOPWORDS
    import matplotlib.pyplot as plt
    %matplotlib inline

    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=1200,height=1500,max_words = 50).generate(text_string_2)


    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    WordCloud.to_file(wordcloud,'word_cloud.jpg')
    return wordcloud

In [ ]:
get_wordcloud(outcome)